# Azure Translator

Azure translator translates text from one language to another using the Translator REST API.


### Overview
*Safety Incident Reports Dataset*: The safety incident reports JSON files in languages like spanish and portuguese are translated to english using the azure translator.

### Notebook Organization 
+ Fetch the injury report JSON files from a folder in spanish and portugese.

+ Translate the JSON files to english by sending a post request to the azure translator service.

+ Store the translated JSON files to a folder.


## Importing Relevant Libraries

In [1]:
import pandas as pd
import requests
import json
import os
from os import listdir
from os.path import isfile, join

## Create Local Folders

- *input-json-files* is the folder from where the input JSON files are provided to be translated into english.
- *translated_json* consists of all the translated json files.

In [2]:
local_input_path = os.path.join(os.getcwd(), "output_json")
# *translated_json* will contain all the translated json files
if (not os.path.isdir(os.getcwd()+"/translated_json")):
    os.makedirs(os.getcwd()+"/translated_json")
output_path = os.path.join(os.getcwd(), "translated_json//")

## Translator Resource

In [3]:
# Importing user defined config
import config

# Translator resource
# Endpoint parameters for querying the translator to return the translated JSON
url = config.TRANLATION_URL
print(url)
headers = config.TRANLATION_HEADERS
print(headers)
# Provide english as the language
lang = "en"

https://Mutarjum-7q65w5v4qwqdm-sep22r2.cognitiveservices.azure.com/translator/text/v3.0/translate?api-version=3.0&to=
{'Ocp-Apim-Subscription-Key': 'dd1b68b1cac449b8a6164bf585323f64', 'Content-Type': 'application/json; charset=UTF-8'}


In [4]:
# Total files in the input folder
files = [f for f in listdir(local_input_path) if isfile(join(local_input_path, f))]
len(files)
# Loop through all the JSON files and translate them one by one
for file in files:
    with open(join(local_input_path,file)) as f:
        # Reading the data from the JSON file
        data = f.readlines()
        data = [json.loads(line) for line in data]
        pd_data = {k.replace('_', ' ') : v for k, v in data[0].items()}
        esp_data = json.dumps(pd_data, ensure_ascii=False).encode('utf8')
        print("\nOriginal JSON\n")
        print(esp_data.decode())
        esp_decod = esp_data.decode('utf-8')
    # Sending a post request to the translator
    resp = requests.post(url+lang, json=[{'Text':esp_decod}], headers = headers)
    resp_text = json.loads(resp.text) 
    en_val=resp_text[0]['translations'][0]['text']
    try:
        en_dict = json.loads(en_val)

    except:
        en_str = f"{{{en_val}}}"
        en_dict = json.loads(en_str)
    print("\nTranslated JSON\n")
    print(en_dict)
    # Save the translated text to a json file
    with open(output_path+file[:-5]+"-translated"+".json", 'w') as outfile:
        json.dump(en_dict, outfile)


Original JSON

{"hospitalized": "1", "event date": "1/1/2015", "final narrative": "A truck driver fell approximately 4 feet while descending a tanker trailer ladder. The employee was kept overnight awaiting test results.", "source": "tractor", "location": "Stuttgart", "caseid": "202045000", "event": "Other fall to lower level less than 6 feet", "amputation": "0", "employer": "Wide World Importers", "part of body": "nonclassifiable", "nature": "traumatic injuries", "form url": "https://stcognitivesearch001.blob.core.windows.net/formupload/202045000.pdf"}

Translated JSON

{'hospitalized': '1', 'event date': '1/1/2015', 'final narrative': 'A truck driver fell approximately 4 feet while descending a tanker trailer ladder. The employee was kept overnight awaiting test results.', 'source': 'tractor', 'location': 'Stuttgart', 'caseid': '202045000', 'event': 'Other fall to lower level less than 6 feet', 'amputation': '0', 'employer': 'Wide World Importers', 'part of body': 'nonclassifiable',